In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

## Gather data

In [2]:
iris = datasets.load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [4]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
target = iris.target
df['target']=target

In [6]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
df.shape

(150, 5)

In [8]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [9]:
mapper = {0:'setosa',1:'versicolor', 2:'virginica'}
df['target'] = df['target'].map(mapper)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Prepare the data for ML

In [10]:
X = df.drop('target', axis=1)

In [11]:
y= df['target']

In [12]:
y.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [13]:
from sklearn.preprocessing import LabelBinarizer

In [14]:
encoder = LabelBinarizer()

In [15]:
y= encoder.fit_transform(y)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [20]:
scaler.fit(X_train)

MinMaxScaler()

In [21]:
scaled_X_train = scaler.transform(X_train)

In [22]:
scaled_X_test = scaler.transform(X_test)

## Build the model

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(units=4, activation='relu', input_shape = [4,]))
model.add(Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])
          


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(patience=10)

In [ ]:
model.fit(x=scaled_X_train, y=y_train, epochs=500,
          validation_data=(scaled_X_test, y_test), verbose=1 ,callbacks=[early_stop]         )

## Evaluation the model

In [ ]:
metrics = pd.DataFrame(model.history.history)
metrics

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
metrics[['accuracy','val_accuracy']].plot()

In [ ]:
model.evaluate(scaled_X_test,y_test,verbose=0)

## Retrain the model with all data

In [ ]:
epochs = len(metrics)

In [ ]:
scaled_X = scaler.fit_transform(X)

In [ ]:
model = Sequential()
model.add(Dense(units=4,activation='relu'))

model.add(Dense(units=3,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(scaled_X,y,epochs=epochs)

## Save the model and the scaler

In [ ]:
model.save("final_iris_model.h5")

In [ ]:
import joblib
joblib.dump(scaler,'iris_scaler.pkl')

## Test and predict

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
iris_model = load_model("final_iris_model.h5")
iris_scaler = joblib.load("iris_scaler.pkl")

In [ ]:
flower_example = {'sepal_length':5.1,
                 'sepal_width':3.5,
                 'petal_length':1.4,
                 'petal_width':0.2}

In [ ]:
encoder.classes_

In [ ]:
def prediction(model,scaler,sample_json):
    
       
    s_len = sample_json['sepal_length']
    s_wid = sample_json['sepal_width']
    p_len = sample_json['petal_length']
    p_wid = sample_json['petal_width']
    
    flower = [[s_len,s_wid,p_len,p_wid]]
    
    flower = scaler.transform(flower)
    
    classes = np.array(['setosa', 'versicolor', 'virginica'])
    
    class_index = np.argmax(model.predict(flower), axis=-1)[0]
    
    return classes[class_index]

In [ ]:
prediction(iris_model,iris_scaler,flower_example)

## Code to be used in text editor

In [ ]:
from tensorflow.keras.models import load_model
import joblib

iris_model = load_model("final_iris_model.h5")
iris_scaler = joblib.load("iris_scaler.pkl")

def prediction(model,scaler,sample_json):
    
       
    s_len = sample_json['sepal_length']
    s_wid = sample_json['sepal_width']
    p_len = sample_json['petal_length']
    p_wid = sample_json['petal_width']
    
    flower = [[s_len,s_wid,p_len,p_wid]]
    
    flower = scaler.transform(flower)
    
    classes = np.array(['setosa', 'versicolor', 'virginica'])
    
    class_index = np.argmax(model.predict(flower), axis=-1)[0]
    
    return classes[class_index]